In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from natsort import natsorted

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
# Vamos definir a pasta que está o dataset
workspace = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\'
root = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\RealWorld\\'

# Lista de usuários e atividades
users = natsorted(os.listdir(root))
tarefas = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking']
SAC = ['sitting', 'standing', 'walking', 'climbingup', 'climbingdown', 'running']
posicao = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
posicaopt = ['peito', 'antebraço', 'cabeça', 'canela', 'coxa', 'braço', 'cintura']

In [4]:
pasta = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\realworld views\\'
D = [None] * 7
D[0] = pd.read_csv(pasta+posicao[0]+'_nova50hz.csv')
D[1] = pd.read_csv(pasta+posicao[1]+'_nova50hz.csv')
D[2] = pd.read_csv(pasta+posicao[2]+'_nova50hz.csv')
D[3] = pd.read_csv(pasta+posicao[3]+'_nova50hz.csv')
D[4] = pd.read_csv(pasta+posicao[4]+'_nova50hz.csv')
D[5] = pd.read_csv(pasta+posicao[5]+'_nova50hz.csv')
D[6] = pd.read_csv(pasta+posicao[6]+'_nova50hz.csv')

In [6]:
900/5*2

360.0

In [41]:
X = D[0].values[:,:900] / 30
y = D[0].values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()
X_test = torch.tensor(X_test, dtype=torch.float32, device=device).detach()
X_val = torch.tensor(X_val, dtype=torch.float32, device=device).detach()

In [91]:
class automlp(nn.Module):
    def __init__(self):
        super().__init__()
        dim = 150
        self.all_layers = torch.nn.Sequential(

            # Encoder
            nn.Flatten(start_dim=1),
            nn.Linear(900, dim),
            # nn.ReLU(),
            nn.LeakyReLU(),
            # nn.Tanh(),

            # Decoder
            nn.Linear(dim, 900)
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits

In [92]:
model = automlp().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 20   # number of epochs to run
batch_size = 150  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)
trainLoss = []
valLoss = []

In [88]:
# AJUSTAR ESSE TREINAMENTO!!
for epoch in range(n_epochs):
    model.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:]
            # forward pass
            Xh = model(X_batch)
            loss = loss_fn(Xh, X_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    Xh = model(X_val)
    valLoss.append(float(loss_fn(Xh, X_val)))
    Xh = model(X_train)
    trainLoss.append(float(loss_fn(Xh, X_train)))

fig = go.Figure()
fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
fig.show()

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 196.10batch/s, loss=0.000334]


In [93]:
# AJUSTAR ESSE TREINAMENTO!!
for epoch in range(n_epochs):
    model.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:]
            # forward pass
            Xh = model(X_batch)
            loss = loss_fn(Xh, X_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    Xh = model(X_val)
    valLoss.append(float(loss_fn(Xh, X_val)))
    Xh = model(X_train)
    trainLoss.append(float(loss_fn(Xh, X_train)))

fig = go.Figure()
fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
fig.show()

Epoch 19: 100%|██████████| 109/109 [00:00<00:00, 168.49batch/s, loss=0.000355]


In [94]:
Xh = model(X_val).cpu().detach().numpy()
aux = X_val.cpu().detach().numpy()
fig = go.Figure()
fig.add_trace(go.Scatter(y=aux[10,:], mode="lines", showlegend=True, name='original'))
fig.add_trace(go.Scatter(y=Xh[10,:], mode="lines", showlegend=True, name='reconstrução'))
fig.show()